# Time constant of the thermistor

In [1]:
using Plots
gr()
include("ThermistorHW.jl")

THW

In [2]:
D = 2e-3
U0 = 10.0
Ts = 90.0
Ta = 25.0
Pa = 93.0


93.0

In [3]:
h = THW.hconvect(U0, D, Ts, Ta, Pa)

249.69368554623864

In [4]:
k = THW.Fe₂O₃.k
ρ = THW.Fe₂O₃.ρ
cₚ = THW.Fe₂O₃.cₚ 


650.6

In [5]:
Bi = D*h/k

0.8610127087801333

In [6]:
R = D/2
A = 4π*R^2
Vol = 4π*R^3/3
m = Vol * ρ

2.195763825349026e-5

In [7]:
t₀ = m*cₚ/(h*A)

4.5528386678250605

In [8]:
θ₀ = 0.1
τ₀ = -log(θ₀)

2.3025850929940455

In [9]:
tconst = τ₀ * t₀

10.483298447340854

In [10]:
t₀


4.5528386678250605

In [12]:
C = t₀ / (D*sqrt(D/U0))

160967.15478337137

In [13]:
tfun(U,D) = 1.42108e5 * D*sqrt(D/U)

tfun (generic function with 1 method)

In [19]:
tfun(U0, 0.2e-3)

0.12710525925861607

In [74]:
D

0.001